In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls


from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, log_loss, classification_report)
from imblearn.over_sampling import SMOTE
import xgboost

import warnings
warnings.filterwarnings('ignore')

In [0]:
attrition = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
attrition.head()

In [0]:
display(attrition.isnull().any())

In [0]:
attrition.Attrition.value_counts().plot(kind='bar',alpha=.65)
plt.title("Attrition Breakdown")

In [0]:
pd.crosstab(attrition.Gender,attrition.Attrition).plot(kind='bar')
plt.title('Attrition with respect to Gender')
plt.xlabel('Gender')
plt.ylabel('Frequency of Attrition')

In [0]:
pd.crosstab(attrition.BusinessTravel,attrition.Attrition).plot(kind='bar')
plt.title('Attrition with respect to BusinessTravel')
plt.xlabel('BusinessTravel')
plt.ylabel('Frequency of Attrition')
plt.xticks(rotation=40)

In [0]:
pd.crosstab(attrition.Department,attrition.Attrition).plot(kind='bar', stacked=False)
plt.title('Attrition with respect to Department')
plt.xlabel('Department')
plt.ylabel('Frequency of Attrition')
plt.xticks(rotation=40)

In [0]:
pd.crosstab(attrition['EducationField'],attrition['Attrition']).plot(kind='bar',stacked=False)
plt.title('Attrition with respect to EducationField')
plt.xlabel('EducationField')
plt.ylabel('Frequency of Attrition')

In [0]:
pd.crosstab(attrition['JobRole'],attrition['Attrition']).plot(kind='bar', stacked=False)
plt.title('Attrition with respect to JobRole')
plt.xlabel('JobRole')
plt.ylabel('Frequency of Attrition')

In [0]:
pd.crosstab(attrition['MaritalStatus'],attrition['Attrition']).plot(kind='bar', stacked=False)
plt.title('Attrition with respect to MaritalStatus')
plt.xlabel('MaritalStatus')
plt.ylabel('Frequency of Attrition')

In [0]:

f, axes = plt.subplots(3, 3, figsize=(10, 8),
sharex=False, sharey=False)

s = np.linspace(0, 3, 10)
cmap = sns.cubehelix_palette(start=0.0, light=1, as_cmap=True)

x = attrition["Age"].values
y = attrition["TotalWorkingYears"].values
sns.kdeplot(x, y, cmap=cmap, shade=True, cut=5, ax=axes[0,0])
axes[0,0].set( title = "Age against Total working years")
cmap = sns.cubehelix_palette(start=0.333333333333, light=1, as_cmap=True)

x = attrition["Age"].values
y = attrition["DailyRate"].values
sns.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[0,1])
axes[0,1].set( title = "Age against Daily Rate")
cmap = sns.cubehelix_palette(start=0.666666666667, light=1, as_cmap=True)

x = attrition["YearsInCurrentRole"].values
y = attrition["Age"].values
sns.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[0,2])
axes[0,2].set( title = "Years in role against Age")
cmap = sns.cubehelix_palette(start=1.0, light=1, as_cmap=True)

x = attrition["DailyRate"].values
y = attrition["DistanceFromHome"].values
sns.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[1,0])
axes[1,0].set( title = "Daily Rate against DistancefromHome")
cmap = sns.cubehelix_palette(start=1.333333333333, light=1, as_cmap=True)

x = attrition["DailyRate"].values
y = attrition["JobSatisfaction"].values
sns.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[1,1])
axes[1,1].set( title = "Daily Rate against Job satisfaction")
cmap = sns.cubehelix_palette(start=1.666666666667, light=1, as_cmap=True)

x = attrition["YearsAtCompany"].values
y = attrition["JobSatisfaction"].values
sns.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[1,2])
axes[1,2].set( title = "Daily Rate against distance")
cmap = sns.cubehelix_palette(start=2.0, light=1, as_cmap=True)

x = attrition["YearsAtCompany"].values
y = attrition["DailyRate"].values
sns.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[2,0])
axes[2,0].set( title = "Years at company against Daily Rate")
cmap = sns.cubehelix_palette(start=2.333333333333, light=1, as_cmap=True)

x = attrition["RelationshipSatisfaction"].values
y = attrition["YearsWithCurrManager"].values
sns.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[2,1])
axes[2,1].set( title = "Relationship Satisfaction vs years with manager")
cmap = sns.cubehelix_palette(start=2.666666666667, light=1, as_cmap=True)

x = attrition["WorkLifeBalance"].values
y = attrition["JobSatisfaction"].values
sns.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[2,2])
axes[2,2].set( title = "WorklifeBalance against Satisfaction")
f.tight_layout()

In [0]:

numerical = [u'Age', u'DailyRate', u'DistanceFromHome', 
             u'Education', u'EmployeeNumber', u'EnvironmentSatisfaction',
             u'HourlyRate', u'JobInvolvement', u'JobLevel', u'JobSatisfaction',
             u'MonthlyIncome', u'MonthlyRate', u'NumCompaniesWorked',
             u'PercentSalaryHike', u'PerformanceRating', u'RelationshipSatisfaction',
             u'StockOptionLevel', u'TotalWorkingYears',
             u'TrainingTimesLastYear', u'WorkLifeBalance', u'YearsAtCompany',
             u'YearsInCurrentRole', u'YearsSinceLastPromotion',u'YearsWithCurrManager']
data = [
    go.Heatmap(
        z= attrition[numerical].astype(float).corr().values, # Generating the Pearson correlation
        x=attrition[numerical].columns.values,
        y=attrition[numerical].columns.values,
        colorscale='Viridis',
        reversescale = False,
#         text = True ,
        opacity = 1.0
        
    )
]


layout = go.Layout(
    title='Pearson Correlation of numerical features',
    xaxis = dict(ticks='', nticks=36),
    yaxis = dict(ticks='' ),
    width = 900, height = 700,
    
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')

In [0]:
attrition_tar= attrition.drop(['Attrition'], axis=1)
y=attrition['Attrition']

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(attrition_tar, 
                                                         y, 
                                                         train_size= 0.80,
                                                         random_state=0);

In [0]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [0]:
type(y_train)

In [0]:
def encoding(data):
   
    categorical = []
    for col, value in data.iteritems():
        if value.dtype == 'object':
            categorical.append(col)
    numerical = data.columns.difference(categorical)
    attrition_cat = data[categorical]
    attrition_cat = pd.get_dummies(attrition_cat)
    attrition_num = data[numerical]
    attrition_final = pd.concat([attrition_num, attrition_cat], axis=1)
    print(attrition_final.columns)
    return attrition_final

In [0]:
x_train_encode=encoding(x_train)
x_train_encode.shape

In [0]:
x_test_encode=encoding(x_test)

In [0]:
def target_encode(data):
    
    target_map = {'Yes':1, 'No':0}
    target = data.apply(lambda x: target_map[x])
    return target

In [0]:
y_train_encode=target_encode(y_train)
y_test_encode=target_encode(y_test)

In [0]:
scalar=StandardScaler()
x_train_scaled=scalar.fit(x_train_encode)
x_test_scaled=scalar.transform(x_train_encode)

In [0]:
clf=SVC()
clf.fit(x_train_encode,y_train_encode)
y_pred=clf.predict(x_test_encode)

In [0]:
accuracy_score(y_test_encode,y_pred)

In [0]:
hparams = {"C": [0.1, 1, 10, 100],
                  "kernel": ["linear", "rbf", "poly"], 
                   "gamma": [0.001, 0.0001]}

In [0]:
grid_search_cv=GridSearchCV(SVC(),param_grid=hparams,scoring='accuracy',n_jobs=-1)
grid_search_cv.fit(x_train_encode,y_train_encode)

In [0]:
print(grid_search_cv.test_score_)
grid_search_cv.best_params_

In [0]:
rdm_search_cv=RandomizedSearchCV(SVC(),param_distributions=hparams,scoring='accuracy',n_jobs=-1)
rdm_search_cv.fit(x_train_encode,y_train_encode)

In [0]:
print(rdm_search_cv.test_score_)
rdm_search_cv.best_params_

In [0]:
oversampler=SMOTE(random_state=0)
smote_train, smote_target = oversampler.fit_sample(x_train_encode,y_train_encode)

In [0]:
seed = 0 
rf_params = {
    'n_jobs': -1,
    'n_estimators': 1000,
#     'warm_start': True, 
    'max_features': 0.3,
    'max_depth': 4,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'random_state' : seed,
    'verbose': 0
}
rf = RandomForestClassifier(**rf_params)
rf.fit(smote_train, smote_target)
print("Fitting of Random Forest finished")

rf_predictions = rf.predict(x_test_encode)
print("Predictions finished")

In [0]:
print("Accuracy score: {}".format(accuracy_score(y_test_encode, rf_predictions)))
print("="*80)
print(classification_report(y_test_encode, rf_predictions))

In [0]:
features = x_train_encode.columns.values
importances = rf.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(15,15))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [0]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
dt.fit(smote_train, smote_target)
print("Fitting of Random Forest finished")

dt_predictions = dt.predict(x_test_encode)
print("Predictions finished")

print("Accuracy score: {}".format(accuracy_score(y_test_encode, dt_predictions)))
print("="*80)
print(classification_report(y_test_encode, dt_predictions))

In [0]:
features = x_train_encode.columns.values
importances = dt.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(15,15))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()


In [0]:
gb_params ={
    'n_estimators': 1500,
    'max_features': 0.9,
    'learning_rate' : 0.25,
    'max_depth': 4,
    'min_samples_leaf': 2,
    'subsample': 1,
    'max_features' : 'sqrt',
    'random_state' : seed,
    'verbose': 0
}

gb = GradientBoostingClassifier(**gb_params)

gb.fit(smote_train, smote_target)

gb_predictions = gb.predict(x_test_encode)
print("Predictions have finished")

print(accuracy_score(y_test_encode, gb_predictions))
print("="*80)
print(classification_report(y_test_encode, gb_predictions))

In [0]:
features = x_train_encode.columns.values
importances = gb.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(15,15))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()